# Apple - User Playlist Retention and Discovery Rates

```SQL
CREATE TABLE users (
    user_id INTEGER,
    user_name VARCHAR
);

CREATE TABLE tracks_added (
    interaction_id INTEGER,
    user_id INTEGER,
    track_id INTEGER,
    added_date DATE,
    is_recommended BOOLEAN
);

INSERT INTO users (user_id, user_name)
VALUES
    (101, 'Alice'),
    (102, 'Bob'),
    (103, 'Charlie'),
    (104, 'Dawn'),
    (105, 'Eve'),
    (106, 'Frank');

INSERT INTO tracks_added (interaction_id, user_id, track_id, added_date, is_recommended)
VALUES
    (1, 101, 1001, '2024-10-02', TRUE),
    (2, 102, 1002, '2024-10-03', TRUE),
    (3, 101, 1003, '2024-10-05', FALSE),
    (4, 103, 1004, '2024-10-06', TRUE),
    (5, 104, 1005, '2024-10-07', TRUE),
    (6, 102, 1006, '2024-10-08', FALSE),
    (7, 101, 1007, '2024-10-09', TRUE),
    (8, 105, 1008, '2024-10-10', TRUE),
    (9, 104, 1009, '2024-10-11', FALSE),
    (10, 106, 1010, '2024-10-12', TRUE),
    (11, 102, 1011, '2024-10-13', TRUE),
    (12, 103, 1012, '2024-10-14', TRUE),
    (13, 104, 1013, '2024-10-15', TRUE),
    (14, 105, 1014, '2024-10-16', TRUE),
    (15, 106, 1015, '2024-10-17', FALSE),
    (16, 101, 1016, '2024-10-18', TRUE),
    (17, 102, 1017, '2024-10-19', TRUE),
    (18, 103, 1018, '2024-10-20', TRUE),
    (19, 104, 1019, '2024-10-21', TRUE),
    (20, 105, 1020, '2024-10-22', TRUE),
    (21, 106, 1021, '2024-10-02', FALSE),
    (22, 101, 1022, '2024-10-03', TRUE),
    (23, 104, 1023, '2024-10-05', TRUE),
    (24, 102, 1024, '2024-10-06', FALSE),
    (25, 105, 1025, '2024-10-07', TRUE),
    (26, 103, 1026, '2024-10-08', TRUE),
    (27, 101, 1027, '2024-10-09', TRUE),
    (28, 106, 1028, '2024-10-10', FALSE),
    (29, 102, 1029, '2024-10-11', TRUE),
    (30, 104, 1030, '2024-10-12', TRUE);
```

In [1]:
import pandas as pd
import numpy as np

In [18]:
df_users = pd.read_csv('Data/007/users_2.csv')
df_tracks = pd.read_csv('Data/007/tracks_added.csv')

df_users.head()

,user_id,user_name
0,101,Alice
1,102,Bob
2,103,Charlie
3,104,Dawn
4,105,Eve


In [3]:
df_tracks.head()

,interaction_id,user_id,track_id,added_date,is_recommended
0,1,101,1001,2024-10-02,True
1,2,102,1002,2024-10-03,True
2,3,101,1003,2024-10-05,False
3,4,103,1004,2024-10-06,True
4,5,104,1005,2024-10-07,True


# Pregunat 1

### ¿Cuántos usuarios únicos han añadido al menos una canción recomendada a sus listas de reproducción en octubre de 2024?

In [15]:
respuesta = df_tracks[

(df_tracks['is_recommended'] == True) &

(df_tracks['added_date'].between('2024-10-01', '2024-10-31'))

]['user_id'].nunique()

respuesta

6

```SQL
SELECT
    count(distinct user_id) AS total_unique_users
FROM tracks_added
WHERE added_date BETWEEN '2024-10-01' AND '2024-10-31'
AND is_recommended = true
```

# Pregunta 2

### Entre los usuarios que añadieron canciones recomendadas en octubre de 2024, ¿cuál es el promedio de canciones recomendadas añadidas a sus listas de reproducción? Por favor, redondea el resultado a 1 decimal para una mejor legibilidad.

In [22]:
df_avg_recommend =  df_tracks[
    (df_tracks['added_date'].between('2024-10-01','2024-10-31')) &
    (df_tracks['is_recommended'] == True)
]

df_avg_count_recommend = df_avg_recommend.groupby('user_id')['is_recommended'].count()

respuesta = df_avg_count_recommend.mean().round(1)

respuesta

np.float64(3.8)

```SQL
SELECT
    ROUND(AVG(avg_recommended),1) AS avg_count_recommended
FROM (
SELECT
    user_id,
    COUNT(*) AS avg_recommended
FROM tracks_added
WHERE added_date BETWEEN '2024-10-01' AND '2024-10-31'
AND is_recommended = true
GROUP BY user_id) AS Subquery;
```

# Pregunta 3

### ¿Podrías darnos el nombre (o nombres) de los usuarios que añadieron una canción no recomendada a su lista de reproducción el 2 de octubre de 2024?

In [25]:
df_merged = pd.merge(df_users, df_tracks, on='user_id')

df_no_recommend = df_merged[
    (df_merged['added_date'] == '2024-10-02') &
    (df_merged['is_recommended'] == False)
]

respuesta3 = df_no_recommend['user_name'].reset_index()

respuesta3

,index,user_name
0,28,Frank


```SQL
SELECT
    u.user_name
FROM users u
JOIN tracks_added t ON u.user_id = t.user_id
WHERE t.added_date = '2024-10-02'
AND t.is_recommended = False;
```